In [ ]:
! kaggle datasets download uciml/sms-spam-collection-dataset
! unzip sms-spam-collection-dataset.zip

  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 71.9MB/s]
Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
sms_data = pd.read_csv('/content/spam.csv', encoding = 'ISO-8859-1')
sms_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
sms_data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [ ]:
sms_data['v1'] = LabelEncoder().fit_transform(sms_data['v1'])

In [ ]:
sms_data.rename({'v1' : 'class', 'v2' : 'text'}, axis = 1, inplace = True)

In [ ]:
sms_data.head()

,class,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels=train_test_split(  sms_data['text'].to_numpy(),
                                                                            sms_data['class'].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=42)

In [ ]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [ ]:
max_vocab_length = 5000
max_length = 16

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
text_vectorize = TextVectorization(max_tokens=max_vocab_length,
                                   output_mode='int',
                                   output_sequence_length=max_length)
text_vectorize.adapt(train_sentences)
embedding = layers.Embedding(input_dim = max_vocab_length,output_dim= 128,input_length = max_length)

In [ ]:
inputs = layers.Input(shape = (1,), dtype = 'string')
x = text_vectorize(inputs)
x = embedding(x)
x  = layers.LSTM(64)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs, name='model_LSTM')

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=50,
                          validation_data=(val_sentences, val_labels),
                          callbacks=[callback]
                         )

Epoch 1/50
140/140 [==============================] - 7s 18ms/step - loss: 0.0672 - accuracy: 0.9798 - val_loss: 0.0935 - val_accuracy: 0.9776
Epoch 2/50
140/140 [==============================] - 1s 11ms/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 0.1024 - val_accuracy: 0.9794
Epoch 3/50
140/140 [==============================] - 1s 10ms/step - loss: 0.0031 - accuracy: 0.9996 - val_loss: 0.0986 - val_accuracy: 0.9776
Epoch 4/50
140/140 [==============================] - 1s 10ms/step - loss: 0.0017 - accuracy: 0.9998 - val_loss: 0.3769 - val_accuracy: 0.9507
Epoch 5/50
140/140 [==============================] - 2s 12ms/step - loss: 0.0113 - accuracy: 0.9960 - val_loss: 0.1709 - val_accuracy: 0.9749
